In [1]:
import pandas as pd
import geopandas as gpd
import shapely

## Population

### Population Import

In [2]:
df = pd.read_csv("../raw_data/density/population/population.csv")

### Cleaning

In [ ]:
df_2000 = df.iloc[0:195]
df_2010 = df.iloc[195:]
df_2010 = df_2010.reset_index()

In [ ]:
df_2010['ntacode'] = df_2010['NTA Code']
df_2010 = df_2010.drop(['index','Borough','Year','FIPS County Code','NTA Name','NTA Code'], axis=1)

### Geocoding

In [3]:
poly = gpd.read_file("../raw_data/density/population/geo_export.shp")

poly['geometry'] = poly.geometry.map(lambda polygon: shapely.ops.transform(lambda x, y: (y, x), polygon))

poly.centroid
poly["NTA_lat"] = poly.centroid.x
poly["NTA_long"] = poly.centroid.y

NTAs = poly.drop(['boro_code', 'boro_name', 'county_fip', 'shape_area', 'shape_leng', 'geometry', 'ntaname'], axis=1)

/tmp/ipykernel_11675/3693915574.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poly.centroid
/tmp/ipykernel_11675/3693915574.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poly["NTA_lat"] = poly.centroid.x
/tmp/ipykernel_11675/3693915574.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poly["NTA_long"] = poly.centroid.y


### Additional Cleaning

In [4]:
population = NTAs.set_index('ntacode').join(df_2010.set_index('ntacode'))
geocoded_population = population.reset_index()
geocoded_population = geocoded_population.drop(['ntacode'], axis=1)

In [5]:
geocoded_population = geocoded_population.replace(to_replace={'Population': {0: df['Population'].median()}}, value=None)

### Population Export

In [6]:
geocoded_population.to_csv('../processed_data/density/geocoded_population.csv', index=False, encoding='utf-8')